# 5. Liveness

## 5.1 Deadlock
Deadlock:

- Each member is waiting for another member to take action

Liveness:

- Properties that require a system to make progress
- Members may have to "take turns" in critical sections

In [1]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 5

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    while sushi_count > 0: # eat sushi until it's all gone
        first_chopstick.acquire()
        second_chopstick.acquire()

        if sushi_count > 0:
            sushi_count -= 1
            print(name, 'took a piece! Sushi remaining:', sushi_count)

        second_chopstick.release()
        first_chopstick.release()

if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_c, chopstick_a)).start()


Barron took a piece! Sushi remaining: 4
Barron took a piece! Sushi remaining: 3
Barron took a piece! Sushi remaining: 2
Barron took a piece! Sushi remaining: 1
Barron took a piece! Sushi remaining: 0


Notice that each philosopher has a different first and second chopstick. 

Due to scheduling, only one philosopher is getting to eat here but all of the sushi is eventually eaten and the program finishes, which is valid behavior. This highlights the tricky nature of deadlocks and why they are hard to detect and debug. Just like a race condition, you might get lucky and never experience a problem with your program, even if the potential for a deadlock exists.

To give this program more opportunities to deadlock, we are going to increase the amount of sushi from 5 to 500, with three really hungry philosophers.

In [2]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 500

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    while sushi_count > 0: # eat sushi until it's all gone
        first_chopstick.acquire()
        second_chopstick.acquire()

        if sushi_count > 0:
            sushi_count -= 1
            print(name, 'took a piece! Sushi remaining:', sushi_count)

        second_chopstick.release()
        first_chopstick.release()

if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_c, chopstick_a)).start()


Barron took a piece! Sushi remaining: 499
Barron took a piece! Sushi remaining: 498
Barron took a piece! Sushi remaining: 497
Barron took a piece! Sushi remaining: 496
Barron took a piece! Sushi remaining: 495
Barron took a piece! Sushi remaining: 494
Barron took a piece! Sushi remaining: 493
Barron took a piece! Sushi remaining: 492
Barron took a piece! Sushi remaining: 491
Barron took a piece! Sushi remaining: 490
Barron took a piece! Sushi remaining: 489
Barron took a piece! Sushi remaining: 488
Barron took a piece! Sushi remaining: 487
Barron took a piece! Sushi remaining: 486


The programs lock up with 486 pieces of sushi remaining. Our philosophers are in a deadlock. If we press Ctrl+Shift+Esc to open the Windows Task Manager and go to the performance tab, we will that the CPU is not overly busy (it's probably only at about 1%). Since the threads are stuck waiting on each other, the deadlock program doesn't use up CPU cycles. Now our program will be stuck in this state forever, so we need to manually terminate it. Running this program again, we will result in a deadlock after a different amount of sushi. The amount of progress it makes before the deadlock will vary depending on how the threads get scheduled to execute. If we're lucky, it's possible that the program could make it to the end of the 500 piece sushi plate. But luck is not a good strategy for programming. So let's implement the solution of prioritizing the locks.

### Prioritize Locks
- Chopstick A = First priority
- Chopstick B = Second priority
- Chopstick A = Third priority

Each philosopher should always acquire their highest priority chopstick first.

In [ ]:
if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_c, chopstick_a)).start()

We can see Steve is causing the problem here because he acquires chopstick C before A. So we need to swap the order of those.

In [3]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 500

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    while sushi_count > 0: # eat sushi until it's all gone
        first_chopstick.acquire()
        second_chopstick.acquire()

        if sushi_count > 0:
            sushi_count -= 1
            print(name, 'took a piece! Sushi remaining:', sushi_count)

        second_chopstick.release()
        first_chopstick.release()

if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_c)).start()

Barron took a piece! Sushi remaining: 499
Barron took a piece! Sushi remaining: 498
Barron took a piece! Sushi remaining: 497
Barron took a piece! Sushi remaining: 496
Barron took a piece! Sushi remaining: 495
Barron took a piece! Sushi remaining: 494
Barron took a piece! Sushi remaining: 493
Barron took a piece! Sushi remaining: 492
Barron took a piece! Sushi remaining: 491
Barron took a piece! Sushi remaining: 490
Barron took a piece! Sushi remaining: 489
Barron took a piece! Sushi remaining: 488
Barron took a piece! Sushi remaining: 487
Barron took a piece! Sushi remaining: 486
Barron took a piece! Sushi remaining: 485
Barron took a piece! Sushi remaining: 484
Barron took a piece! Sushi remaining: 483
Barron took a piece! Sushi remaining: 482
Barron took a piece! Sushi remaining: 481
Barron took a piece! Sushi remaining: 480
Barron took a piece! Sushi remaining: 479
Barron took a piece! Sushi remaining: 478
Barron took a piece! Sushi remaining: 477
Barron took a piece! Sushi remaini

Olivia took a piece! Sushi remaining: 39
Olivia took a piece! Sushi remaining: 38
Olivia took a piece! Sushi remaining: 37
Olivia took a piece! Sushi remaining: 36
Olivia took a piece! Sushi remaining: 35
Olivia took a piece! Sushi remaining: 34
Olivia took a piece! Sushi remaining: 33
Olivia took a piece! Sushi remaining: 32
Olivia took a piece! Sushi remaining: 31
Olivia took a piece! Sushi remaining: 30
Olivia took a piece! Sushi remaining: 29
Olivia took a piece! Sushi remaining: 28
Olivia took a piece! Sushi remaining: 27
Olivia took a piece! Sushi remaining: 26
Olivia took a piece! Sushi remaining: 25
Olivia took a piece! Sushi remaining: 24
Olivia took a piece! Sushi remaining: 23
Olivia took a piece! Sushi remaining: 22
Olivia took a piece! Sushi remaining: 21
Olivia took a piece! Sushi remaining: 20
Olivia took a piece! Sushi remaining: 19
Olivia took a piece! Sushi remaining: 18
Olivia took a piece! Sushi remaining: 17
Olivia took a piece! Sushi remaining: 16
Olivia took a pi

This time the program runs to the end and we have 3 very well-fed philosophers. We designed this example to be as simple as possible by only including a single shared resource, the sushi plate. In practice, this program only really needs one lock to protect it. 

We intentionally created the potential for deadlock because we nested two locks inside each other to demonstrate the concept. As your program grows in complexity to include more critical sections, locks and parallel threads with intertwined dependencies, it becomes increasingly more difficult to identify and prevent potential deadlocks. 

### Lock Ordering
- The simplest technique to prevent deadlocks
- Ensure locks are always taken in the same order by any thread.

However, lock ordering may not always be feasible. For example, a thread may not know all of the locks it will need to acquire ahead of taking any of them. 


### Lock Timeout
- Put a timeout on lock attempts
- If a thread cannot acquire all locks within the time limit:
    - Back up and free all locks then
    - Wait for a random amount of time
    - Try again!